## Reúso de camadas pré-treinadas e transferência do aprendizado

In [1]:
# Python ≥3.5 é necessário
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 é necessário
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 é necessário
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

import numpy as np
import os

np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### Reusando um modelo da biblioteca Keras

Vamos dividir o conjunto de dados MNIST em dois:
* `X_train_A`: todas as imagens do conjunto de dados, exceto sandálias e camisas (classes 5 e 6).
* `X_train_B`: conjunto de treinamento menor com as 200 primeiras imagens de sandálias e camisetas.

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandálias ou camisas
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # índices das classes 7, 8, 9 deveriam ser alterados para 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # classificação binária: é uma camisa? (classe 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [3]:
X_train_A.shape

(43986, 28, 28)

In [4]:
X_train_B.shape

(200, 28, 28)

In [5]:
tf.random.set_seed(42)
np.random.seed(42)

In [6]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [7]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [8]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.5923 - accuracy: 0.8091 - val_loss: 0.3815 - val_accuracy: 0.8647
Epoch 2/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3532 - accuracy: 0.8780 - val_loss: 0.3222 - val_accuracy: 0.8859
Epoch 3/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.3150 - accuracy: 0.8911 - val_loss: 0.2958 - val_accuracy: 0.8989
Epoch 4/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2956 - accuracy: 0.8986 - val_loss: 0.2816 - val_accuracy: 0.9041
Epoch 5/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.2825 - accuracy: 0.9028 - val_loss: 0.2730 - val_accuracy: 0.9091
Epoch 6/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.2728 - accuracy: 0.9072 - val_loss: 0.2658 - val_accuracy: 0.9106
Epoch 7/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2647 - accuracy: 0.9096 - val_loss: 0.2633 - val_accuracy:

In [9]:
model_A.save("my_model_A.h5")

In [10]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [11]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [12]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 36ms/step - loss: 0.7333 - accuracy: 0.5950 - val_loss: 0.4617 - val_accuracy: 0.8093
Epoch 2/20
7/7 [==============================] - 0s 14ms/step - loss: 0.4079 - accuracy: 0.8450 - val_loss: 0.3692 - val_accuracy: 0.8641
Epoch 3/20
7/7 [==============================] - 0s 14ms/step - loss: 0.3244 - accuracy: 0.8900 - val_loss: 0.3096 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 15ms/step - loss: 0.2705 - accuracy: 0.9150 - val_loss: 0.2660 - val_accuracy: 0.9260
Epoch 5/20
7/7 [==============================] - 0s 15ms/step - loss: 0.2308 - accuracy: 0.9350 - val_loss: 0.2334 - val_accuracy: 0.9402
Epoch 6/20
7/7 [==============================] - 0s 15ms/step - loss: 0.2004 - accuracy: 0.9500 - val_loss: 0.2083 - val_accuracy: 0.9513
Epoch 7/20
7/7 [==============================] - 0s 15ms/step - loss: 0.1776 - accuracy: 0.9600 - val_loss: 0.1908 - val_accuracy: 0.9544
Epoch 8/20
7/7 [===========

In [13]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [14]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [15]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 1s 47ms/step - loss: 0.3058 - accuracy: 0.8950 - val_loss: 0.3448 - val_accuracy: 0.8682
Epoch 2/16
7/7 [==============================] - 0s 19ms/step - loss: 0.2953 - accuracy: 0.9050 - val_loss: 0.3352 - val_accuracy: 0.8773
Epoch 3/16
7/7 [==============================] - 0s 18ms/step - loss: 0.2863 - accuracy: 0.9150 - val_loss: 0.3258 - val_accuracy: 0.8854
Epoch 4/16
7/7 [==============================] - 0s 18ms/step - loss: 0.2776 - accuracy: 0.9200 - val_loss: 0.3167 - val_accuracy: 0.8905
Epoch 5/16
7/7 [==============================] - 0s 18ms/step - loss: 0.2692 - accuracy: 0.9200 - val_loss: 0.3080 - val_accuracy: 0.8935
Epoch 6/16
7/7 [==============================] - 0s 18ms/step - loss: 0.2611 - accuracy: 0.9350 - val_loss: 0.2997 - val_accuracy: 0.9016
Epoch 7/16
7/7 [==============================] - 0s 18ms/step - loss: 0.2535 - accuracy: 0.9350 - val_loss: 0.2921 - val_accuracy: 0.9047
Epoch 8/16
7/7 [===========

In [16]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 1s 53ms/step - loss: 0.1857 - accuracy: 0.9550 - val_loss: 0.1997 - val_accuracy: 0.9442
Epoch 2/16
7/7 [==============================] - 0s 21ms/step - loss: 0.1516 - accuracy: 0.9600 - val_loss: 0.1736 - val_accuracy: 0.9513
Epoch 3/16
7/7 [==============================] - 0s 29ms/step - loss: 0.1294 - accuracy: 0.9700 - val_loss: 0.1527 - val_accuracy: 0.9564
Epoch 4/16
7/7 [==============================] - 0s 23ms/step - loss: 0.1120 - accuracy: 0.9750 - val_loss: 0.1366 - val_accuracy: 0.9625
Epoch 5/16
7/7 [==============================] - 0s 23ms/step - loss: 0.0984 - accuracy: 0.9800 - val_loss: 0.1246 - val_accuracy: 0.9696
Epoch 6/16
7/7 [==============================] - 0s 25ms/step - loss: 0.0879 - accuracy: 0.9850 - val_loss: 0.1147 - val_accuracy: 0.9726
Epoch 7/16
7/7 [==============================] - 0s 25ms/step - loss: 0.0793 - accuracy: 0.9850 - val_loss: 0.1070 - val_accuracy: 0.9746
Epoch 8/16
7/7 [===========

### Acurácia dos modelos:

In [17]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0936 - accuracy: 0.9810


[0.0935618132352829, 0.9810000061988831]

In [18]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0692 - accuracy: 0.9840


[0.06920021027326584, 0.984000027179718]